# 1. dataset 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
import optuna
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager

# sklearn 관련
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
import statsmodels.api as sm 
from time import time

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


# 전처리 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# lightgbm 관련
from lightgbm import LGBMRegressor
from lightgbm import plot_importance

/Users/yj.noh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 한글 폰트 경로 설정
font_path = '/System/Library/Fonts/AppleSDGothicNeo.ttc'
font_name = font_manager.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

In [2]:
data = pd.read_csv('combined_data.csv', encoding = "cp949")
data.head()

#data = data[data['pick_rgn2_nm']=='강남구']
print(data.shape) 

(60000, 13)


In [3]:
# Checking for null values
print(data.info())

# Checking for outliers
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pick_rgn2_nm   60000 non-null  object 
 1   rider_cnt      60000 non-null  float64
 2   datetime       60000 non-null  object 
 3   hour_reg       60000 non-null  int64  
 4   reg_date       60000 non-null  object 
 5   day_of_reg     60000 non-null  object 
 6   is_holiday     60000 non-null  int64  
 7   is_rain        60000 non-null  int64  
 8   rider_cnt_w_1  60000 non-null  float64
 9   rider_cnt_w_2  60000 non-null  float64
 10  rider_cnt_w_3  60000 non-null  float64
 11  rider_cnt_w_4  60000 non-null  float64
 12  order_cnt_w_1  60000 non-null  int64  
dtypes: float64(5), int64(4), object(4)
memory usage: 6.0+ MB
None
          rider_cnt     hour_reg    is_holiday      is_rain  rider_cnt_w_1   
count  60000.000000  60000.00000  60000.000000  60000.00000   60000.000000  \
mean    

In [4]:
data["datetime"] = pd.to_datetime(data["datetime"])
data["reg_date"] = pd.to_datetime(data["reg_date"])

data = data.sort_values(by="datetime")

In [ ]:
# data = data.drop(columns = ['is_crm'])
# print(data.head())

In [5]:
# data = data.dropna(subset=['rider_cnt_w_4'])
data.isna().sum()
#print(data.shape) 

pick_rgn2_nm     0
rider_cnt        0
datetime         0
hour_reg         0
reg_date         0
day_of_reg       0
is_holiday       0
is_rain          0
rider_cnt_w_1    0
rider_cnt_w_2    0
rider_cnt_w_3    0
rider_cnt_w_4    0
order_cnt_w_1    0
dtype: int64

In [6]:
# category  - pick_rgn2_nm, hour_reg, day_of_reg, is_rain, month, week, is_holiday
for col in ['day_of_reg','pick_rgn2_nm', 'hour_reg', 'is_holiday', 'is_rain'] : 
    data[col] = data[col].astype('category')

print(data.dtypes)


pick_rgn2_nm           category
rider_cnt               float64
datetime         datetime64[ns]
hour_reg               category
reg_date         datetime64[ns]
day_of_reg             category
is_holiday             category
is_rain                category
rider_cnt_w_1           float64
rider_cnt_w_2           float64
rider_cnt_w_3           float64
rider_cnt_w_4           float64
order_cnt_w_1             int64
dtype: object


# 2. 데이터 전처리

In [ ]:
# numeric 변수 scale 
# scaler = StandardScaler()  #평균 0 , 분산 1로 조정
# #scaler = MinMaxScaler()

# num_vars = ['rider_cnt_2', 'rider_cnt_w_1', 'rider_cnt_w_2', 'rider_cnt_w_3',
#             'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2', 'order_cnt_w_3',
#             'order_cnt_w_4']
# data[num_vars] = scaler.fit_transform(data[num_vars])

# print(df.head(3))

## 2-1. one-hot-encoding

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))

In [8]:
#df = data.drop(columns = ['rider_cnt_w_3','rider_cnt_w_4'])
df = data
var = ['day_of_reg','pick_rgn2_nm', 'hour_reg', 'is_holiday', 'is_rain']
encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(data[var]).toarray(), columns=encoder.get_feature_names_out(var), index = data.index)
df = pd.concat([onehot, df.drop(columns=var)], axis=1)
#print(df.head(3))
print(df.columns)

Index(['day_of_reg_금요일', 'day_of_reg_목요일', 'day_of_reg_수요일', 'day_of_reg_월요일',
       'day_of_reg_일요일', 'day_of_reg_토요일', 'day_of_reg_화요일',
       'pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
       'pick_rgn2_nm_강서구', 'pick_rgn2_nm_관악구', 'pick_rgn2_nm_광진구',
       'pick_rgn2_nm_구로구', 'pick_rgn2_nm_금천구', 'pick_rgn2_nm_노원구',
       'pick_rgn2_nm_도봉구', 'pick_rgn2_nm_동대문구', 'pick_rgn2_nm_동작구',
       'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
       'pick_rgn2_nm_성동구', 'pick_rgn2_nm_성북구', 'pick_rgn2_nm_송파구',
       'pick_rgn2_nm_양천구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_용산구',
       'pick_rgn2_nm_은평구', 'pick_rgn2_nm_종로구', 'pick_rgn2_nm_중구',
       'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_10', 'hour_reg_11',
       'hour_reg_12', 'hour_reg_13', 'hour_reg_14', 'hour_reg_15',
       'hour_reg_16', 'hour_reg_17', 'hour_reg_18', 'hour_reg_19',
       'hour_reg_20', 'hour_reg_21', 'hour_reg_22', 'hour_reg_23',
       'is_holiday_0', 'is_holiday_1', 'is_rain_0'

# 3. train, test set split

In [9]:
# train_ratio = 0.8
# total_samples = df.shape[0]
# train_samples = int(train_ratio * total_samples)
# df_train = df[:train_samples]
# df_test = df[train_samples:]

df_train = df[df["reg_date"]<= '2023-04-30']
df_test = df[df["reg_date"] >= '2023-05-01']

print(df_train['reg_date'].min()) #2023-01-04
print(df_train['reg_date'].max()) #2023-04-30
print(df_test['reg_date'].min()) #2023-05-01
print(df_test['reg_date'].max()) #2023-06-12

df_train = df_train.drop(columns = ['datetime', 'reg_date'])
df_test = df_test.drop(columns = ['datetime', 'reg_date'])
print(df_train.shape, df_test.shape) # 43875, 16125 

2023-01-04 00:00:00
2023-04-30 00:00:00
2023-05-01 00:00:00
2023-06-12 00:00:00
(43875, 57) (16125, 57)


In [10]:
# X_train, y_train 나누기
 
# X_train = train.iloc[:, :-1]
# y_train = df_train.iloc[:, -1]

# X_test = df_test.iloc[:, :-1]
# y_test = df_test.iloc[:, -1]

X_train = df_train.drop(columns=['rider_cnt'])
y_train = df_train['rider_cnt']

X_test = df_test.drop(columns=['rider_cnt'])
y_test = df_test['rider_cnt']

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(43875, 56) (43875,) (16125, 56) (16125,)


In [11]:
print(X_train.columns)

Index(['day_of_reg_금요일', 'day_of_reg_목요일', 'day_of_reg_수요일', 'day_of_reg_월요일',
       'day_of_reg_일요일', 'day_of_reg_토요일', 'day_of_reg_화요일',
       'pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
       'pick_rgn2_nm_강서구', 'pick_rgn2_nm_관악구', 'pick_rgn2_nm_광진구',
       'pick_rgn2_nm_구로구', 'pick_rgn2_nm_금천구', 'pick_rgn2_nm_노원구',
       'pick_rgn2_nm_도봉구', 'pick_rgn2_nm_동대문구', 'pick_rgn2_nm_동작구',
       'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
       'pick_rgn2_nm_성동구', 'pick_rgn2_nm_성북구', 'pick_rgn2_nm_송파구',
       'pick_rgn2_nm_양천구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_용산구',
       'pick_rgn2_nm_은평구', 'pick_rgn2_nm_종로구', 'pick_rgn2_nm_중구',
       'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_10', 'hour_reg_11',
       'hour_reg_12', 'hour_reg_13', 'hour_reg_14', 'hour_reg_15',
       'hour_reg_16', 'hour_reg_17', 'hour_reg_18', 'hour_reg_19',
       'hour_reg_20', 'hour_reg_21', 'hour_reg_22', 'hour_reg_23',
       'is_holiday_0', 'is_holiday_1', 'is_rain_0'

### numeric_scale 

In [ ]:
# # 입력 변수 
# numeric_cols = ['rider_cnt_w_1', 'rider_cnt_w_2', 'rider_cnt_w_3',
#                 'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2', 'order_cnt_w_3',
#                 'order_cnt_w_4']

# # scaler 
# scaler_X = StandardScaler()

# # X_train, X_test
# X_train_scaled = scaler_X.fit_transform(X_train[numeric_cols])
# X_test_scaled = scaler_X.transform(X_test[numeric_cols])

# # 스케일링된 결과를 DataFrame으로 변환
# X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_cols, index = X_train.index)
# X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_cols, index = X_test.index)

# # 원래의 범주형 변수들을 선택
# categorical_cols = [col for col in X_train.columns if col not in numeric_cols]
# X_train_cat = X_train[categorical_cols]
# X_test_cat = X_test[categorical_cols]

# # 스케일링된 DataFrame과 범주형 변수들을 병합
# X_train_final = pd.concat([X_train_scaled, X_train_cat], axis=1)
# X_test_final = pd.concat([X_test_scaled, X_test_cat], axis=1)



In [ ]:
# 예측값
# y_train, y_test
# scaler_y = StandardScaler()
# y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
# y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))
# y_train_scaled = y_train_scaled.ravel()
# y_test_scaled=  y_test_scaled.ravel()

# print(y_train_scaled.shape)
# print(y_test_scaled.shape)

# 3. regression - benchmark model

In [12]:
X_train_lm = sm.add_constant(X_train)

lr_1 = sm.OLS(y_train, X_train_lm).fit()

print(lr_1.summary())

                            OLS Regression Results                            
Dep. Variable:              rider_cnt   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 2.669e+04
Date:                Tue, 13 Jun 2023   Prob (F-statistic):               0.00
Time:                        23:01:56   Log-Likelihood:            -2.0758e+05
No. Observations:               43875   AIC:                         4.153e+05
Df Residuals:                   43823   BIC:                         4.157e+05
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 6.3289      0.33

In [ ]:
# 선형 회귀 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 잔차 계산
y_pred = model.predict(X_test)
residuals = y_test - y_pred

# 변수별 잔차 그래프 그리기
for column in X_test.columns:
    plt.figure(figsize=(10,6))
    sns.scatterplot(x=X_test[column], y=residuals)
    plt.title(f'Residuals vs. {column}')
    plt.xlabel(column)
    plt.ylabel('Residuals')
    plt.show()


# 4.Machine Learning Modeling

## 4-1. 하이퍼파라미터 튜닝 - Grid Search 

### a. LightGBM model 

In [ ]:
classifier = LGBMRegressor()

parameters = [{'learning_rate': [0.1, 0.05, 0.01, 0.005], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]},
              {'learning_rate': [0.15, 0.125, 0.1, 0.075], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7], 'num_leaves': [16, 32, 64]}]

grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
                           cv=10,
                           n_jobs=-1,
                           refit='neg_mean_squared_error')                     

grid_search.fit(X_train, y_train)
best_rmse = np.sqrt(-1 * grid_search.cv_results_['mean_test_neg_mean_squared_error'][grid_search.best_index_])
best_mae = -1 * grid_search.cv_results_['mean_test_neg_mean_absolute_error'][grid_search.best_index_]
best_parameters = grid_search.best_params_
print("Best RMSE: {:.2f}".format(best_rmse))
print("Best MAE: {:.2f}".format(best_mae))
print("Best Parameters:", best_parameters)

# best rmse : 27,91
# best mae : 17.55
# Best Parameters: {'learning_rate': 0.075, 'max_depth': 7, 'n_estimators': 50, 'num_leaves': 64}

In [ ]:
def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "num_leaves": trial.suggest_int("num_leaves", 16, 64),
    }

    model = LGBMRegressor(**params)
    
    score = cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
    rmse = np.sqrt(-1 * np.mean(score))

    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best RMSE: {:.2f}".format(study.best_value))
print("Best Parameters:", study.best_params)

#Best RMSE: 33.40
#Best Parameters: {'learning_rate': 0.0170925353196432, 'n_estimators': 187, 'max_depth': 3, 'num_leaves': 24}

### b. ridge regression

In [ ]:
# Ridge Regression
ridge = Ridge()
ridge_param_grid = {'alpha': [0.1, 1.0, 2.0, 5.0, 10.0]}
ridge_grid_search = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
ridge_grid_search.fit(X_train, y_train)
print("Ridge Best RMSE: {:.2f}".format(np.sqrt(-ridge_grid_search.best_score_)))
print("Ridge Best Parameters: ", ridge_grid_search.best_params_)

# Ridge Best RMSE: 27.83
# Ridge Best Parameters:  {'alpha': 10.0}

### c. Lasso regression

In [ ]:
# Lasso Regression
lasso = Lasso(max_iter = 10000)
lasso_param_grid = {'alpha': [0.1, 1.0,2.0, 5.0, 10.0]}
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
lasso_grid_search.fit(X_train, y_train)
print("Lasso Best RMSE: {:.2f}".format(np.sqrt(-lasso_grid_search.best_score_)))
print("Lasso Best Parameters: ", lasso_grid_search.best_params_)

# Lasso Best RMSE: 28.83
# Lasso Best Parameters:  {'alpha': 0.1}

### d. Support vector regressor

In [ ]:
# SVR
# svr = SVR()
# svr_param_grid = {'kernel': ['rbf', 'linear', 'poly', 'sigmoid'], 'C': [0.1, 1.0, 10.0], 'gamma': ['scale', 'auto']}
# svr_grid_search = GridSearchCV(estimator=svr, param_grid=svr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
# svr_grid_search.fit(df_X, df_y)
# print("SVR Best RMSE: {:.2f}".format(np.sqrt(-svr_grid_search.best_score_)))
# print("SVR Best Parameters: ", svr_grid_search.best_params_)

### e. Random Forest Regressor

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        'min_samples_split': [2, 5, 10],
    }

    model = RandomForestRegressor(**params)
    
    score = cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
    rmse = np.sqrt(-1 * np.mean(score))

    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best RMSE: {:.2f}".format(study.best_value))
print("Best Parameters:", study.best_params)


In [ ]:
# Random Forest Regressor
rfr = RandomForestRegressor(random_state=0)
rfr_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7, 9], 'min_samples_split': [2, 5, 10]}
rfr_grid_search = GridSearchCV(estimator=rfr, param_grid=rfr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
rfr_grid_search.fit(X_train, y_train)
print("Random Forest Regressor Best RMSE: {:.2f}".format(np.sqrt(-rfr_grid_search.best_score_)))
print("Random Forest Regressor Best Parameters: ", rfr_grid_search.best_params_)

# Random Forest Regressor Best RMSE: 28.9
# Random Forest Regressor Best Parameters:  {'max_depth': 9, 'min_samples_split': 2, 'n_estimators': 200}


### f. Decision Tree Regressor

In [ ]:
# Decision Tree Regressor
dtr = DecisionTreeRegressor(random_state=0)
dtr_param_grid = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]}
dtr_grid_search = GridSearchCV(estimator=dtr, param_grid=dtr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
dtr_grid_search.fit(X_train, y_train)
print("Decision Tree Regressor Best RMSE: {:.2f}".format(np.sqrt(-dtr_grid_search.best_score_)))
print("Decision Tree Regressor Best Parameters: ", dtr_grid_search.best_params_)

# Decision Tree Regressor Best RMSE: 27.85
# Decision Tree Regressor Best Parameters:  {'max_depth': 7, 'min_samples_split': 2}

## 4-2. train, test set 적용 

### a. train, test rmse, mae

In [13]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) /y_test)) *100

In [14]:
train_set = data[data["datetime"] <= '2023-04-30']
test_set = data[data["datetime"] >= '2023-05-01']

def execute_pipeline(X_train, y_train, X_test, y_test):
    regressors = [
        Lasso(alpha=0.1, max_iter=5000),
        LGBMRegressor(learning_rate =  0.0170925353196432, n_estimators =  187, max_depth =  3, num_leaves = 24),
        RandomForestRegressor(random_state=0, max_depth=9, min_samples_split=2, n_estimators=200),
    ]

    result_test = pd.DataFrame({'datetime': test_set["reg_date"],
                                'pick_rgn2_nm': test_set["pick_rgn2_nm"], 'hour_reg': test_set["hour_reg"], 
                                'is_rain': test_set["is_rain"], 'day_of_reg': test_set["day_of_reg"], "is_holiday" : test_set["is_holiday"],
                                'y_test': test_set["rider_cnt"]})
    
    scores = {}
    predictions = {}

    for reg in regressors:
        reg_name = reg.__class__.__name__
        scoring = {
            'rmse' : 'neg_root_mean_squared_error',
            'mae' : 'neg_mean_absolute_error',
            'r2' : 'r2'
        } 
        
        cv_results = cross_validate(reg, X_train, y_train, cv = 10, scoring = scoring)
        
        # Cross-validation RMSE and SD
        cv_rmse = -np.mean(cv_results['test_rmse'])
        cv_rmse_std = np.std(cv_results['test_rmse'])

        reg.fit(X_train, y_train)
        y_pred_test = reg.predict(X_test)
        
        rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae_test = mean_absolute_error(y_test, y_pred_test)
        mape_test = MAPE(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)

        # 모델 저장
        model_file = f'model_{reg_name}.joblib'
        joblib.dump(reg, model_file)

        scores[reg_name] = {
            'cv_rmse' : cv_rmse,
            'cv_rmse_std' : cv_rmse_std,
            'Test RMSE': rmse_test,
            'Test MAE': mae_test,
            'Test MAPE': mape_test,
            'Test R2': r2_test
        }
         
        result_test[f'y_pred_test_{reg_name}'] = y_pred_test
             
        predictions[reg_name] = y_pred_test

    lasso_pred = predictions['Lasso']
    lgbm_pred = predictions['LGBMRegressor']
    rf_pred = predictions['RandomForestRegressor']
    average_three = (lasso_pred + lgbm_pred + rf_pred) / 3
    average_lgbm_rf = (lgbm_pred+rf_pred) /2 
    average_lgbm_la = (lgbm_pred + lasso_pred) /2
    average_rf_la = (lasso_pred+rf_pred) / 2

    result_test['y_pred_avg_three'] = average_three
    result_test['y_pred_avg_lgbm_rf'] = average_lgbm_rf
    result_test['y_pred_avg_lgbm_la'] = average_lgbm_la
    result_test['y_pred_avg_rf_la'] = average_rf_la

    scores_df = pd.DataFrame(scores).transpose()

    # train, test 예측치 저장
    result_test.to_csv('prediction_results_test_set_w_1.csv', index=False, encoding="cp949")

    return scores_df

scores_df = execute_pipeline(X_train, y_train, X_test, y_test)
print(scores_df)


                         cv_rmse  cv_rmse_std  Test RMSE   Test MAE   
Lasso                  30.110331    16.257629  36.654935  22.628781  \
LGBMRegressor          31.084848    13.020167  38.650055  23.497868   
RandomForestRegressor  30.143973    15.544332  36.394466  22.148548   

                       Test MAPE   Test R2  
Lasso                  10.940498  0.947684  
LGBMRegressor          12.180137  0.941834  
RandomForestRegressor  10.411454  0.948425  


In [15]:
# Calculate MAE, RMSE and MAPE for each model
predict = pd.read_csv('prediction_results_test_set.csv', encoding = "cp949")

metrics = ['MAE', 'RMSE', 'MAPE']
models = ['test_Lasso', 'test_LGBMRegressor', 'test_RandomForestRegressor', 'avg_three', 'avg_lgbm_rf', 'avg_lgbm_la', 'avg_rf_la']

results = []

for model in models:
    y_true = predict['y_test']
    y_pred = predict[f'y_pred_{model}']

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = MAPE(y_true, y_pred)

    results.append({
        'model': model,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    })

result_df = pd.DataFrame(results)
print(result_df)


                        model        MAE       RMSE      MAPE
0                  test_Lasso  18.882900  29.807232  9.562142
1          test_LGBMRegressor  19.220658  31.150254  9.563269
2  test_RandomForestRegressor  19.126212  30.928254  9.517928
3                   avg_three  18.820776  30.323411  9.365396
4                 avg_lgbm_rf  19.064186  30.912108  9.475467
5                 avg_lgbm_la  18.818944  30.190033  9.392240
6                   avg_rf_la  18.773283  30.095528  9.367487


### 세분화하여 MAE 값 확인

In [42]:
# 데이터 프레임에서 각 모델의 예측 값 - 실제 값 계산
predict = pd.read_csv('prediction_results_test_set.csv', encoding = "cp949")


predict['MAE_Lasso'] = abs(predict['y_pred_test_Lasso'] - predict['y_test'])
predict['MAE_LGBMRegressor'] = abs(predict['y_pred_test_LGBMRegressor'] - predict['y_test'])
predict['MAE_RandomForestRegressor'] = abs(predict['y_pred_test_RandomForestRegressor'] - predict['y_test'])

predict['MAE_avg_three'] = abs(predict['y_pred_avg_three'] - predict['y_test'])
predict['MAE_avg_lgbm_rf'] = abs(predict['y_pred_avg_lgbm_rf'] - predict['y_test'])
predict['MAE_avg_lgbm_la'] = abs(predict['y_pred_avg_lgbm_la'] - predict['y_test'])
predict['MAE_avg_rf_la'] = abs(predict['y_pred_avg_rf_la'] - predict['y_test'])

result = predict.groupby(['hour_reg']).agg({
    'MAE_Lasso': np.mean,
    'MAE_LGBMRegressor': np.mean,
    'MAE_RandomForestRegressor': np.mean,
    'MAE_avg_three' : np.mean,
    'MAE_avg_lgbm_rf' : np.mean,
    'MAE_avg_lgbm_la' : np.mean,
    'MAE_avg_rf_la' : np.mean
    
}).reset_index()

print(result)



    hour_reg  MAE_Lasso  MAE_LGBMRegressor  MAE_RandomForestRegressor   
0          9   7.266325           7.219739                   7.215067  \
1         10  12.185760          12.330130                  12.549569   
2         11  19.099050          19.668999                  19.656040   
3         12  22.500644          22.739935                  22.952975   
4         13  22.674797          23.142457                  22.824233   
5         14  18.259554          18.902962                  18.961445   
6         15  20.682500          21.120941                  20.887604   
7         16  22.765041          23.200235                  22.993995   
8         17  25.404444          25.580909                  25.104571   
9         18  23.909732          24.654076                  24.525722   
10        19  24.227704          25.029970                  25.013878   
11        20  21.420359          22.197540                  22.147409   
12        21  16.959376          17.418873         

### 변수 중요도 파악 

### permutation importance -> RandomForest, LGBM

In [ ]:
# permutation importance 

def calculate_permutation_importance(model, X_train, y_train, X_test, y_test):
    # train set에 대한 permutation importance 계산
    perm_importance_train = permutation_importance(model, X_train, y_train, n_repeats=5, random_state=42)

    # test set에 대한 permutation importance 계산
    perm_importance_test = permutation_importance(model, X_test, y_test, n_repeats=5, random_state=42)

    # importance 값 저장 (numpy array를 list로 변환)
    perm_importances_train = perm_importance_train.importances_mean.tolist()
    perm_importances_test = perm_importance_test.importances_mean.tolist()

    # 특성명과 importance 값 매핑
    feature_importances = pd.DataFrame({
        'feature': X_train.columns,
        'perm_importance_train': perm_importances_train,
        'perm_importance_test': perm_importances_test,
    })

    # importance 값에 따라 내림차순 정렬
    feature_importances = feature_importances.sort_values(by='perm_importance_train', ascending=False)

    # CSV 파일로 저장
    feature_importances.to_csv('feature_importances_lgbm.csv', index=False, encoding="cp949")
    
    return feature_importances

# LGBM 모델을 불러옴
lgbm_model = joblib.load('model_LGBMRegressor.joblib')

# permutation importance 계산
feature_importances = calculate_permutation_importance(lgbm_model, X_train, y_train, X_test, y_test)

#print(scores_df)
print(feature_importances)

In [ ]:
# permutation importance 

def calculate_permutation_importance(model, X_train, y_train, X_test, y_test):
    perm_importance_train = permutation_importance(model, X_train, y_train, n_repeats=5, random_state=42)
    
    perm_importance_test = permutation_importance(model, X_test, y_test, n_repeats=5, random_state=42)

    perm_importances_train = perm_importance_train.importances_mean.tolist()
    perm_importances_test = perm_importance_test.importances_mean.tolist()

    # 특성명과 importance 값 매핑
    feature_importances = pd.DataFrame({
        'feature': X_train.columns,
        'perm_importance_train': perm_importances_train,
        'perm_importance_test': perm_importances_test,
    })

    feature_importances = feature_importances.sort_values(by='perm_importance_train', ascending=False)
    
    feature_importances.to_csv('feature_importances_Rf.csv', index=False, encoding="cp949")
    
    return feature_importances

lgbm_model = joblib.load('model_RandomForestRegressor.joblib')

feature_importances = calculate_permutation_importance(lgbm_model, X_train, y_train, X_test, y_test)

print(feature_importances)

In [ ]:
lasso_model = joblib.load('model_Lasso.joblib')
lasso_coef = pd.Series(lasso_model.coef_, index=X_train.columns)
selected_feats_lasso = lasso_coef[lasso_coef!=0].index

print("Number of features selected by Lasso: ", len(selected_feats_lasso))
print("Features selected by Lasso: ", selected_feats_lasso)


In [ ]:
def plot_feature_importances(model, model_name):
    importances = model.feature_importances_
    feat_names = X_train.columns
    feature_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)

    #print("Feature importances for ", model_name, " : ")
    #print(feature_imp)

    # Creating a bar plot
    plt.figure(figsize = (20,15))
    sns.barplot(x=feature_imp, y=feature_imp.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()

lgbm_model = joblib.load('model_LGBMRegressor.joblib')
plot_feature_importances(lgbm_model, 'LGBMRegressor')

rf_model = joblib.load('model_RandomForestRegressor.joblib')
plot_feature_importances(rf_model, 'RandomForestRegressor')


### stacking model 

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Initialize models
lasso = Lasso()
lr = LinearRegression()
rf = RandomForestRegressor()

# Meta model
gbr = GradientBoostingRegressor()

# Stacking model
stack = StackingCVRegressor(regressors=(lasso, lr, rf),
                            meta_regressor=gbr,
                            use_features_in_secondary=True)

# Training the stacking classifier
stack.fit(X_train.values, y_train.values)

# Predict
y_pred = stack.predict(X_test.values)

# Evaluate the model
print('Stack Test RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
